In [1]:
import openpyxl as xl
import pandas as pd
import os
from time import sleep


In [5]:
files_dir = r"C:\Users\Matheus\Documents\Git\Pontte\automacaoRegistroAztronic\automacaoRegistroCCI\Simuladores"
simuladorFiles = [f for f in os.listdir(files_dir) if f.endswith(".xlsm")]
simuladorFiles

['FI Simulador Juliana.xlsm',
 'FI Simulador Romilton.xlsm',
 'HE Simulador Agostinho.xlsm',
 'He Simulador Conrado.xlsm',
 'HE Simulador Margareide.xlsm',
 'HE Simulador Patricia.xlsm']

In [73]:
path = os.path.join(files_dir, simuladorFiles[0])
row2 = 2
simuladorFiles[0]

'FI Simulador Juliana.xlsm'

In [74]:
wb = xl.load_workbook(path, data_only=True)
ws3 = wb['Simulador PDF']

c:\Users\Matheus\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [102]:
#Coletar Informações do Quadro Simulação
amortizacao = None
indexador = None
prazo = None
carencia = None
pularParcela = None
diaVencimento = None
taxaAM = None
CET = None

#Coletar Informações do Quadro Resultados
LTV = None
IOF = None
TAC = None
registro = None
valorBruto = None
valorLiquido = None
valorAvaliacao = None
primeiraParcela = None

#Auxiliar Primeira Parcela
auxPar = False
dataDesembolco = None
priParcelaAmortizao = None
priParcelaJuros = None
dataUltimaParcela = 0

for col in ws3.iter_cols(min_col=1,max_col=15, min_row = 1, max_row=100):
    for cell in col:
        if cell.value == 'Simulação':
            #Definir variáveis auxiliares de linha e coluna
            rowAux = cell.row + 1
            columnAux = cell.column + 3

            #Extração da tabela simulação
            amortizacao = ws3.cell(row=rowAux, column=columnAux).value
            indexador = ws3.cell(row=rowAux+1, column=columnAux).value
            prazo = ws3.cell(row=rowAux+2, column=columnAux).value
            carencia = ws3.cell(row=rowAux+3, column=columnAux).value
            pularParcela = ws3.cell(row=rowAux+4, column=columnAux).value
            diaVencimento = ws3.cell(row=rowAux+5, column=columnAux).value
            taxaAM = ws3.cell(row=rowAux+6, column=columnAux).value
            CET = ws3.cell(row=rowAux+7, column=columnAux).value
        
        elif cell.value == 'Resultados':
            #Definir variáveis auxiliares de linha e coluna
            rowAux = cell.row + 1
            columnAux = cell.column + 3

            #Extralção da tabela Resultados
            LTV = ws3.cell(row=rowAux, column=columnAux).value
            IOF = ws3.cell(row=rowAux+1, column=columnAux).value
            TAC = ws3.cell(row=rowAux+2, column=columnAux).value
            registro = ws3.cell(row=rowAux+3, column=columnAux).value
            valorBruto = ws3.cell(row=rowAux+4, column=columnAux).value
            valorLiquido = ws3.cell(row=rowAux+5, column=columnAux).value
            valorAvaliacao = ws3.cell(row=rowAux+6, column=columnAux).value
            primeiraParcela = ws3.cell(row=rowAux+7, column=columnAux).value
        
        elif cell.value == 'Data':
            rowAux = cell.row + 1
            columnAux = cell.column + 1
            dataDesembolco = ws3.cell(row=rowAux, column=cell.column).value
            for i in range(1,10):
                parcelaMensal = ws3.cell(row=rowAux+i, column=columnAux).value
                if parcelaMensal > 0 and auxPar == False:
                    priParcelaAmortizao = ws3.cell(row=rowAux+i, column=columnAux+1).value
                    priParcelaJuros = ws3.cell(row=rowAux+i, column=columnAux+2).value
                    #print(f"Parcela do mês: {parcelaMensal} \nAmortização: {priParcelaAmortizao}\nJuros: {priParcelaJuros}\nAmortização+Juros: {str(priParcelaJuros+priParcelaAmortizao)}")
                    auxPar = True
        
        elif cell.value == 'Saldo Devedor':
            for i in range(1,400):
                saldoDevedor = ws3.cell(row=rowAux+i, column=cell.column).value
                if saldoDevedor == None:
                    saldoDevedor = 10
                if saldoDevedor <= 0:
                    dataUltimaParcela = ws3.cell(row=rowAux+i, column=cell.column-4).value

amortizacaoEJuros = priParcelaAmortizao + priParcelaJuros
prestacoes = prazo - carencia
nomeCliente = ws3.cell(row=1, column=7).value
wb.close()

**Conferência dos Valores Extraídos**

In [103]:
#Printar Informações do Quadro Simulação
print(f"Valores Extraídos do quado de Simulação")
print(f"Amoritzação: {amortizacao} \nIndexador: {indexador} \nPrazo: {prazo}\nCarência: {carencia}\nPular Parcela: {pularParcela}\nDia de Vencimento: {diaVencimento}\nTaxa ao mês {taxaAM}\nCET: {CET}")

#Printar Informações do Quadro Resultados
print(f"\n\nValores Extraídos do quado de Resultados")
print(f"Data Desembolço: {dataDesembolco}\nLTV: {LTV}\nIOF: {IOF}\nTAC: {TAC}\nRegistro: {registro}\nValor Bruto: {valorBruto}\nValor Líquido: {valorLiquido}\nValor de Avaliação: {valorAvaliacao}\nValor Primeira Parcela: {primeiraParcela}")

#Auxiliar Primeira Parcela
print(f"\n\nValores Extraídos da Primeira Parcela")
print(f"Valor de Amortização: {priParcelaAmortizao}\nValor do Juros: {priParcelaJuros}\nAmortização+Juros: {str(priParcelaJuros+priParcelaAmortizao)}")

print(f"Prazo: {prestacoes}")

Valores Extraídos do quado de Simulação
Amoritzação: Price 
Indexador: IPCA 
Prazo: 360
Carência: 1
Pular Parcela: Nenhum
Dia de Vencimento: 15
Taxa ao mês 0.007
CET: 0.10564897347453095


Valores Extraídos do quado de Resultados
Data Desembolço: 2022-07-08 00:00:00
LTV: 0.698176581089543
IOF: 0
TAC: 35000
Registro: 79988
Valor Bruto: 1114988
Valor Líquido: 1000000
Valor de Avaliação: 1597000
Valor Primeira Parcela: 8948.353998592136


Valores Extraídos da Primeira Parcela
Valor de Amortização: 700.9358249321394
Valor do Juros: 7874.539194352652
Amortização+Juros: 8575.475019284791
Prazo: 359


In [ ]:
listaDePara = {'valorTotal':valorBruto,'tabela': amortizacao,'registro': registro,'valorLiquido': valorLiquido,'prazoMes': prazo,'valorPrimeiraParcelaComEncargos':primeiraParcela,
                'valorImóvel':valorAvaliacao,'prazoContrato': prestacoes,'ultimaParcela':dataUltimaParcela,'dataContrato': dataDesembolco, 'valorPrimeiraParcela': a
                a,'primeiraParcela':'VENCIMENTO DA PRIMEIRA PRESTAÇÃO :','indice':'ÍNDICE: MENSAL do'
                }